# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    

In [1]:
import findspark
findspark.init('/home/dhrumpy/spark-3.0.0-bin-hadoop3.2')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('churn').getOrCreate()

data = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [2]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [23]:
data.describe().show()

+-------+-----------------+-----------------+-----------------+------------------+-------------------+
|summary|              Age|   Total_Purchase|            Years|         Num_Sites|              Churn|
+-------+-----------------+-----------------+-----------------+------------------+-------------------+
|  count|              900|              900|              900|               900|                900|
|   mean|41.81666666666667|10062.82403333334| 5.27315555555555| 8.587777777777777|0.16666666666666666|
| stddev|6.127560416916251|2408.644531858096|1.274449013194616|1.7648355920350969| 0.3728852122772358|
|    min|             22.0|            100.0|              1.0|               3.0|                  0|
|    max|             65.0|         18026.01|             9.15|              14.0|                  1|
+-------+-----------------+-----------------+-----------------+------------------+-------------------+



In [3]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [12]:
data.select(['Age', 'Total_Purchase', 'Years', 'Num_Sites', 'Churn']).show(5)

+----+--------------+-----+---------+-----+
| Age|Total_Purchase|Years|Num_Sites|Churn|
+----+--------------+-----+---------+-----+
|42.0|       11066.8| 7.22|      8.0|    1|
|41.0|      11916.22|  6.5|     11.0|    1|
|38.0|      12884.75| 6.67|     12.0|    1|
|42.0|       8010.76| 6.71|     10.0|    1|
|37.0|       9191.58| 5.56|      9.0|    1|
+----+--------------+-----+---------+-----+
only showing top 5 rows



In [16]:
# data.groupBy('Company').count().show(5)
data = data.select(['Age', 'Total_Purchase', 'Years', 'Num_Sites', 'Churn']).na.drop()

In [17]:
# NO VECTOR INDEXER REQUIRED HERE AS NO CATEGORICAL VARIABLES PRESENT HERE!
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Years', 'Num_Sites'], outputCol='features')

from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

log_reg_churn = LogisticRegression(featuresCol='features', labelCol='Churn')
pipeline = Pipeline(stages=[assembler,log_reg_churn]) # last entry is the actual model
train_churn_data, test_churn_data = data.randomSplit([0.7,0.3])
fit_model = pipeline.fit(train_churn_data)

In [20]:
results = fit_model.transform(test_churn_data)
results.select('Churn','prediction').show(5)

+-----+----------+
|Churn|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 5 rows



In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')
AUC = my_eval.evaluate(results)
AUC

0.7520661157024794

In [33]:
# Without Pipeline

In [27]:
# dir(assembler)

In [30]:
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Years', 'Num_Sites'], outputCol='features')
output = assembler.transform(data)
output.show(5)

+----+--------------+-----+---------+-----+--------------------+
| Age|Total_Purchase|Years|Num_Sites|Churn|            features|
+----+--------------+-----+---------+-----+--------------------+
|42.0|       11066.8| 7.22|      8.0|    1|[42.0,11066.8,7.2...|
|41.0|      11916.22|  6.5|     11.0|    1|[41.0,11916.22,6....|
|38.0|      12884.75| 6.67|     12.0|    1|[38.0,12884.75,6....|
|42.0|       8010.76| 6.71|     10.0|    1|[42.0,8010.76,6.7...|
|37.0|       9191.58| 5.56|      9.0|    1|[37.0,9191.58,5.5...|
+----+--------------+-----+---------+-----+--------------------+
only showing top 5 rows



In [37]:
final_data = output.select(['features', 'Churn'])
train_churn_data, test_churn_data = final_data.randomSplit([0.7,0.3])
log_reg_churn = LogisticRegression(featuresCol='features', labelCol='Churn')

fit_model = log_reg_churn.fit(train_churn_data)

# results = fit_model.transform(test_churn_data)
# results.select('Churn','prediction').show(5)

In [40]:
fit_model.summary.predictions.show(5)

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[25.0,9672.03,5.4...|  0.0|[4.26893268692309...|[0.98619648972642...|       0.0|
|[26.0,8787.39,5.4...|  1.0|[0.65651176293813...|[0.65847636962466...|       0.0|
|[26.0,8939.61,4.5...|  0.0|[5.90624225048043...|[0.99728499354820...|       0.0|
|[27.0,8628.8,5.3,...|  0.0|[5.47474917849617...|[0.99582621382396...|       0.0|
|[28.0,8670.98,3.9...|  0.0|[7.29661056265639...|[0.99932262678915...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



In [41]:
fit_model.summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                619|                619|
|   mean|0.15508885298869143|0.11147011308562198|
| stddev|0.36228211868144927| 0.3149679240798625|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [42]:
# Evaluation metrics

In [46]:
pred_and_labels = fit_model.evaluate(test_churn_data)
pred_and_labels.predictions.show(5)

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,4....|    0|[4.67528209792083...|[0.99076321874243...|       0.0|
|[28.0,11128.95,5....|    0|[4.32620993417586...|[0.98695487649217...|       0.0|
|[28.0,11204.23,3....|    0|[1.46172566526619...|[0.81179646890756...|       0.0|
|[29.0,5900.78,5.5...|    0|[4.04733983517418...|[0.98283113642063...|       0.0|
|[29.0,10203.18,5....|    0|[3.92395933308240...|[0.98062030245029...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



In [48]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')
AUC = churn_eval.evaluate(pred_and_labels.predictions)
AUC

0.6909365312449014